In [1]:
from sklearn.model_selection import train_test_split
import json
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import keras
from keras import layers
import pandas as pd
import pickle
from IPython.display import clear_output
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


clear_output()

# Trenowanie modelu
W tym dokumencie Jupyter Notebook znajduje się:
- załadowanie danych oraz etykiet
- podział na zbiór treningowy i testowy
- skalowanie danych
- wyszukanie odpowiednich parametrów
- trening modelu
- test modelu z wypisaniem wskaźników jakości modelu
- zapisanie modelu głównego oraz modelu skalującego dane do pliku

### Załadowanie danych oraz etykiet z pliku
Nasze dane posiadają następujące atrybuty:
- cena
- zniżka
- miasto (hot-one)
- kategoria (hot-one)
- czas jaki upłynął od ostatniego zdarzenia w sesji dla tego użytkownika (0 dla pierwszego zdarzenia)
- ile razy dany użytkownik wcześniej wyświetlił ten produkt
- ile razy dany użytkownik wcześniej wyświetlił produkt z tej kategorii
- ile razy dany użytkownik kupił już produkt kiedy zaoferowana była mu zniżka taka jak aktualna lub niższa
- ile razy dany użytkownik kupił jakikolwiek produkt w danej kategorii

In [2]:
with open('../3-preprocess/records-merged.json') as file:
    sessions = json.load(file)

with open('../3-preprocess/buys2.json') as file:
    buys = json.load(file)

features = pd.DataFrame(sessions)
labels = pd.DataFrame(buys)

### Podział na zbiór treningowy i testowy
- użycie wielkości zbioru testowego 0.25 całości zbioru danych
- użycie deterministycznych wyników za pomocą *random_state* oraz *shuffle=False*

In [3]:
RANDOM_STATE = 55
TEST_SIZE = 0.25

X_train, X_test, y_train, y_test = train_test_split(features, labels,
                                                    test_size=TEST_SIZE,
                                                    shuffle=False,
                                                    random_state=RANDOM_STATE)

attrs_count = X_train.shape[1]
X_train.head()

,price,product_views_count,category_view_count,bought_with_lower_count,bought_category_count,discount,time,one_hot_city0,one_hot_city1,one_hot_city2,...,one_hot_category5,one_hot_category6,one_hot_category7,one_hot_category8,one_hot_category9,one_hot_category10,one_hot_category11,one_hot_category12,one_hot_category13,one_hot_category14
0,64.80,0,0,0,0,0,0.0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2048.50,0,0,0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7639.00,0,1,0,0,0,189.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1998.14,0,2,0,0,0,72.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2399.00,0,3,0,0,0,142.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Skalowanie danych
- "karmienie" modelu skalera danymi treningowymi
- transformacja danych testowych i produkcyjnych za pomocą nauczonego modelu skalera

In [4]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.468988,-1.004891,-1.044236,-0.410782,-0.560587,-0.704942,-1.211978,-0.329268,-0.397157,-0.451412,...,-0.157655,-0.108852,6.018668,-0.163763,-0.391228,-0.113742,-0.272901,-0.235988,-0.170299,-0.108852
1,0.909266,-1.004891,-1.044236,-0.410782,-0.560587,-0.704942,-1.211978,-0.329268,-0.397157,-0.451412,...,-0.157655,-0.108852,-0.166150,-0.163763,-0.391228,-0.113742,-0.272901,-0.235988,-0.170299,-0.108852
2,4.793485,-1.004891,-0.989052,-0.410782,-0.560587,-0.704942,0.583690,-0.329268,-0.397157,-0.451412,...,-0.157655,-0.108852,-0.166150,-0.163763,-0.391228,-0.113742,-0.272901,-0.235988,-0.170299,-0.108852
3,0.874276,-1.004891,-0.933867,-0.410782,-0.560587,-0.704942,-0.527914,-0.329268,-0.397157,-0.451412,...,-0.157655,-0.108852,-0.166150,-0.163763,-0.391228,-0.113742,-0.272901,-0.235988,-0.170299,-0.108852
4,1.152789,-1.004891,-0.878683,-0.410782,-0.560587,-0.704942,0.137148,-0.329268,-0.397157,-0.451412,...,-0.157655,-0.108852,-0.166150,-0.163763,-0.391228,-0.113742,-0.272901,-0.235988,-0.170299,-0.108852


### Utworzenie modelu Uczenia Maszynowego
- użycie Keras do tworzenia modelu
- model zawiera 3 warstwy gęste "full connected" o ilości neuronów odpowiednio (100 lub 120), (10, 20 lub 30) oraz 1
- model jako argument przyjmuje rekord złożony z 30 atrybutów opisujących 1 wydarzenie (event) w systemie e-commerce
- model zwraca prawdopodobieństwo kupienia produktu przez użytkownika w formacie liczby [0,1]

In [5]:
def create_model(units1, units2):
    model = keras.Sequential()
    model.add(layers.Dense(units1, activation='relu', input_shape=(attrs_count,)))
    model.add(layers.Dense(units2, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    return model

### Automatyczne strojenie hiperparametrów
- użycie GridSearch do znalezienia wielkości warstw modelu
- użycie wrapera scikit-learn na modele napisane w Keras, aby przekazać je modelowi GridSearch

In [ ]:
keras_clf = KerasClassifier(build_fn=create_model,batch_size=25)

tuned_parameters = [{
    'units1': [100,120],
    'units2': [10,20,30],
    }]

clf = GridSearchCV(keras_clf, tuned_parameters, cv=5)

### Uczenie modelu
- model uczy się wiele razy, aby sprawdzić wszystkie parametry i wybiera najlepiej nauczony model
- wynikowe najlepsze parametry sa podane poniżej

In [ ]:
clf.fit(X_train, y_train)
clear_output()
clf.best_params_

### Test modelu

In [ ]:
y_pred = clf.predict(X_test)
y_pred = [x[0] for x in y_pred]

pd.DataFrame(y_pred).head()

### Wypisanie wskaźników jakości modelu
- dane modelu są niezrównoważone - większość etykiet jest równa zeru -> stosowany jest więc "balanced accuracy score"
- wyniki tablicy pomyłek porównujemy z wynikami modelu bazowego, który zawsze zwraca najczęstszą klasę, czyli 0 (nie kupi)

In [ ]:
print("Balanced accuracy score: ")
print(balanced_accuracy_score(y_test.values.tolist(), y_pred))

In [ ]:
print("Confusion matrix: ")
print(confusion_matrix(y_test, y_pred))
print("Base confusion matrix: ")
print(confusion_matrix(y_test, [0 for _ in range(len(y_test))]))

### Zapisanie modeli do plików

In [ ]:
par = clf.best_params_
model = create_model(par['units1'], par['units2'])
model.fit(X_train, y_train)

with open ('../5-server/core/model.pckl', 'wb') as file:
    pickle.dump(model,file)

with open ('../5-server/core/scaler.pckl', 'wb') as file:
    pickle.dump(sc,file)

In [6]:
par = clf.best_params_
model = create_model(par['units1'], par['units2'])
model.fit(X_train, y_train)

with open ('../5-server/core/model.pckl', 'wb') as file:
    pickle.dump(model,file)

with open ('../5-server/core/scaler.pckl', 'wb') as file:
    pickle.dump(sc,file)

### Uczenie modelu
- model uczy się wiele razy, aby sprawdzić wszystkie parametry i wybiera najlepiej nauczony model
- wynikowe najlepsze parametry sa podane poniżej

In [7]:
clf.fit(X_train, y_train)
clear_output()
clf.best_params_

{'units1': 120, 'units2': 30}

### Test modelu

In [8]:
y_pred = clf.predict(X_test)
y_pred = [x[0] for x in y_pred]

pd.DataFrame(y_pred).head()

,0
0,0
1,0
2,0
3,0
4,0


### Wypisanie wskaźników jakości modelu
- dane modelu są niezrównoważone - większość etykiet jest równa zeru -> stosowany jest więc "balanced accuracy score"
- wyniki tablicy pomyłek porównujemy z wynikami modelu bazowego, który zawsze zwraca najczęstszą klasę, czyli 0 (nie kupi)

In [9]:
print("Balanced accuracy score: ")
print(balanced_accuracy_score(y_test.values.tolist(), y_pred))

Balanced accuracy score: 
0.6223335879215317


In [13]:
print("Confusion matrix: ")
print(confusion_matrix(y_test, y_pred))
print("Base confusion matrix: ")
print(confusion_matrix(y_test, [0 for _ in range(len(y_test))]))

Confusion matrix: 
[[11103    79]
 [  324   109]]
Base confusion matrix: 
[[11182     0]
 [  433     0]]


### Zapisanie modeli do plików

In [11]:
par = clf.best_params_
model = create_model(par['units1'], par['units2'])
model.fit(X_train, y_train)

with open ('../5-server/core/model.pckl', 'wb') as file:
    pickle.dump(model,file)

with open ('../5-server/core/scaler.pckl', 'wb') as file:
    pickle.dump(sc,file)

Epoch 1/1
34842/34842 [==============================] - 3s 96us/step - loss: 0.1204 - acc: 0.9625